In [1]:
def solveSudoku(board: list[list[str]]) -> None:

    def is_valid_placement(board, row, col, num):
        for i in range(9):
            if board[row][i] == num:
                return False
            if board[i][col] == num:
                return False
        
        start_row = row - row % 3
        start_col = col - col % 3
        for i in range(3):
            for j in range(3):
                if board[start_row + i][start_col + j] == num:
                    return False
        return True

    def solve(board):
        for i in range(9):
            for j in range(9):
                if board[i][j] == '.':
                    for num_char in "123456789":
                        if is_valid_placement(board, i, j, num_char):
                            board[i][j] = num_char
                            if solve(board):
                                return True
                            board[i][j] = '.'
                    return False
        return True

    solve(board)

The LeetCode problem 37, "Sudoku Solver," is a classic, challenging problem that requires filling in the empty cells (represented by '.') on a $9 \times 9$ Sudoku board to make the final configuration valid. A valid Sudoku board must satisfy the constraints that every row, every column, and every $3 \times 3$ sub-box contains the digits '1' through '9' exactly once. The problem guarantees that the given partially filled board has only one unique solution.

---

### **The Fundamental Approach: Backtracking**

Due to the combinatorial nature of Sudoku solving—where placing a number in one cell constrains the possibilities for many other cells—the most effective and standard algorithm for this problem is **Backtracking**, implemented via Depth-First Search (DFS). Backtracking systematically explores all potential placements of numbers. When a placement leads to an invalid state or a dead end, the algorithm "backtracks" by undoing the last placement and trying the next available choice.

---

### **The State and Recursion**

The recursive function, typically named `solve()`, attempts to fill the board starting from the first empty cell it finds. The state of the search is defined by the current configuration of the board. The function has a base case and a recursive step:

1.  **Base Case:** If the function has iterated through all cells and found no empty cells, it means the board is completely filled. We check if this final board is valid (though the placement logic should ensure validity). If complete, we return `True` to signal a successful solution.
2.  **Recursive Step:** If an empty cell $(r, c)$ is found, we iterate through all possible digits from '1' to '9'. For each digit:
    * **Check Validity:** We first check if placing the digit $d$ at $(r, c)$ is currently valid according to the Sudoku rules (row, column, and $3 \times 3$ box).
    * **Placement:** If the placement is valid, we place the digit $d$ on the board and recursively call `solve()` for the next empty cell.
    * **Success/Failure:** If the recursive call returns `True` (meaning a full solution was found downstream), we return `True`. If the recursive call returns `False` (meaning $d$ leads to a dead end), we must **backtrack**: we revert the cell $(r, c)$ back to '.' (empty) and try the next possible digit.

---

### **Optimizing Validity Checks (The Data Structures)**

To make the validity check fast ($O(1)$ lookup time), it is highly inefficient to scan the entire board or a row/column/box every time a digit is tested. Instead, we use a set of data structures, typically **Boolean arrays or Hash Sets**, to pre-track which digits are already present in each row, column, and box.

1.  **`rows[9][9]`:** A 2D array where `rows[r][d]` is `True` if digit $d+1$ is present in row $r$.
2.  **`cols[9][9]`:** A 2D array where `cols[c][d]` is `True` if digit $d+1$ is present in column $c$.
3.  **`boxes[9][9]`:** A 2D array where `boxes[b][d]` is `True` if digit $d+1$ is present in box $b$.

These structures are initialized by scanning the original board once before the backtracking starts. The crucial box index $b$ for a cell $(r, c)$ is calculated as: $b = (r/3) \times 3 + (c/3)$. During the backtracking, updating these structures (setting the corresponding entry to `True` for placement and `False` for backtracking) is $O(1)$. 

---

### **Pruning and Search Order**

The efficiency of the solver is dramatically improved by **pruning** the search tree (the constraint checking) and by using an optimized search order.

1.  **Constraint Pruning:** The check for validity *before* the recursive call (e.g., `isValid(r, c, d)`) is the primary form of pruning, as it prevents the algorithm from exploring branches that are known to be invalid immediately.
2.  **Most Constrained Variable (MCV) Heuristic:** A further optimization is to prioritize filling the cell that has the **fewest possible valid options** first. This is known as the Most Constrained Variable heuristic. By dealing with the hardest choices first, we increase the likelihood of discovering an invalid branch early, leading to faster pruning and a quicker solution. While not strictly required for the unique solution guarantee, it significantly boosts performance.

---

### **Complexity Analysis**

The exact worst-case time complexity of Sudoku solving is difficult to state precisely but is often considered exponential without the aid of heuristics. However, for a standard $9 \times 9$ board with unique solution guarantees, the complexity is largely dominated by the size of the search space.

* **Worst-Case Time Complexity (Theoretical):** The number of possible partially filled Sudoku boards is massive. In practice, the constraint propagation and pruning drastically reduce the search time. For typical problems, the complexity is well within acceptable limits.
* **Space Complexity:** The space used is primarily for the $3 \times 9 \times 9$ Boolean arrays/sets used for the constraint tracking, plus the recursion stack depth (at most 81 calls deep). This results in an $O(N^2)$ space complexity, which is $O(81)$, effectively $O(1)$ for a fixed $9 \times 9$ board size.